<a href="https://colab.research.google.com/github/Mateus-Brandao/Estudo/blob/main/Machine_Learning_Valida%C3%A7%C3%A3o_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Dividindo os dados do modelo em treino e teste

In [ ]:
x = dados[['preco','idade_do_modelo','km_por_ano']]
y = dados['vendido']

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, test_size = 0.25, stratify = y)

print('Temos %d amostras de treino e %d amostras de teste' % (len(treino_x), len(teste_x)))

Temos 7500 amostras de treino e 2500 amostras de teste


# Testando a porcentagem de acerto de um método aleatório com o DummyClassifier

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print('A acurácia do dummy stratified foi de %.2f%%' % acuracia)

A acurácia do dummy stratified foi de 58.00%


# Testando o método de Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('A acurácia foi de %.2f%%' % acuracia)

A acurácia foi de 71.92%


In [ ]:
SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, test_size = 0.25, stratify = y)

print('Temos %d amostras de treino e %d amostras de teste' % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth = 2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('A acurácia foi de %.2f%%' % acuracia)

Temos 7500 amostras de treino e 2500 amostras de teste
A acurácia foi de 76.84%


# **Aplicando Cross Validation**
O cross validation serve pra testar várias vezes com conjuntos de treino e testes diferentes e tirar uma média dos valores obtidos, garantindo uma maior confiabilidade de previsão

In [ ]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

# testando com cv = 3
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('A média é de %.2f' % media)
print('O desvio padrão é %.4f' % desvio_padrao)
print('A acurácia é de %.2f%% a %.2f%%' % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

A média é de 0.76
O desvio padrão é 0.0040
A acurácia é de 74.99% a 76.57%


In [ ]:
# testando com cv = 5
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('A média é de %.2f' % media)
print('O desvio padrão é %.4f' % desvio_padrao)
print('A acurácia é de %.2f%% a %.2f%%' % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

A média é de 0.76
O desvio padrão é 0.0029
A acurácia é de 75.21% a 76.35%


In [ ]:
# testando com cv = 10
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score = False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('A média é de %.2f' % media)
print('O desvio padrão é %.4f' % desvio_padrao)
print('A acurácia é de %.2f%% a %.2f%%' % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

A média é de 0.76
O desvio padrão é 0.0077
A acurácia é de 74.24% a 77.32%


# **Adicionando aleatoriedade no Cross Validate**

In [ ]:
def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print('A média é de %.2f' % (media * 100))
  print('O desvio padrão é %.4f' % (desvio_padrao * 100))
  print('A acurácia é de %.2f%% a %.2f%%' % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

# adicionando um gerador de splits aleatório com o kfold
cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score = False)
imprime_resultados(results)

A média é de 75.76
O desvio padrão é 1.2484
A acurácia é de 73.26% a 78.26%


# Simulando uma situação horrível de azar na distribuição de dados

In [ ]:
dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco','idade_do_modelo','km_por_ano']]
y_azar = dados_azar['vendido']
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [ ]:
SEED = 301
np.random.seed(SEED)

# adicionando um gerador de splits aleatório com o kfold
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

A média é de 57.84
O desvio padrão é 11.7736
A acurácia é de 34.29% a 81.39%


In [ ]:
SEED = 301
np.random.seed(SEED)

# adicionando um gerador de splits aleatório com o kfold
cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

A média é de 75.78
O desvio padrão é 1.7423
A acurácia é de 72.30% a 79.26%


# Utilizando o Stratified KFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

A média é de 75.78
O desvio padrão é 1.1161
A acurácia é de 73.55% a 78.01%


In [ ]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score = False)
imprime_resultados(results)

A média é de 75.78
O desvio padrão é 0.9734
A acurácia é de 73.83% a 77.73%


# Criando uma coluna aleatória de modelo dos carros pro dataframe

In [ ]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados['idade_do_modelo'] + np.random.randint(-2,3, size = 10000)

In [ ]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [ ]:
dados['modelo_aleatorio'].unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

In [ ]:
dados['modelo_aleatorio'] = dados['modelo_aleatorio'] + abs(dados['modelo_aleatorio'].min()) + 1
dados['modelo_aleatorio'].unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [ ]:
dados['modelo'] = dados['modelo_aleatorio']
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,18,18
1,40557.96,1,20,12622.05362,24,24
2,89627.50,0,12,11440.79806,14,14
3,95276.14,0,3,43167.32682,6,6
4,117384.68,1,4,12770.11290,5,5


In [ ]:
dados.drop('modelo_aleatorio', inplace = True, axis = 1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


# **Utilizando um estimador para amostras que vão ser criadas no futuro**


In [ ]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results)

A média é de 75.78
O desvio padrão é 1.0562
A acurácia é de 73.67% a 77.90%


# Cross Validate com StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('A acuracia foi de %.2f%%' % acuracia)

A acuracia foi de 77.48%


In [ ]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results)

A média é de 77.27
O desvio padrão é 1.4618
A acurácia é de 74.35% a 80.20%


# Criando uma pipeline para executar o processo do jeito correto

In [ ]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score = False)
imprime_resultados(results)

A média é de 76.68
O desvio padrão é 1.2015
A acurácia é de 74.28% a 79.08%
